#Processamento de Linguagem Natuaral
##Previsão do tipo da reclamação a partir do texto escrito pelo consumidor

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
data=pd.read_csv('/content/drive/MyDrive/Github/data-science-pos-graduation-Uni7/Módulo 15 - Aprendizagem de Maquina - Analytics/job-1-complaints/complaints.csv')

In [8]:
data.shape

(1959680, 18)

In [9]:
categorias = data['Product'].unique()
categorias

array(['Debt collection',
       'Credit reporting, credit repair services, or other personal consumer reports',
       'Credit card or prepaid card', 'Vehicle loan or lease',
       'Money transfer, virtual currency, or money service',
       'Checking or savings account', 'Mortgage', 'Student loan',
       'Payday loan, title loan, or personal loan', 'Credit card',
       'Consumer Loan', 'Payday loan', 'Bank account or service',
       'Credit reporting', 'Other financial service', 'Money transfers',
       'Prepaid card', 'Virtual currency'], dtype=object)

In [10]:
data.isnull().sum()

Date received                         0
Product                               0
Sub-product                      235165
Issue                                 0
Sub-issue                        598707
Consumer complaint narrative    1294836
Company public response         1138807
Company                               0
State                             36141
ZIP code                         188156
Tags                            1709156
Consumer consent provided?       682466
Submitted via                         0
Date sent to company                  0
Company response to consumer          2
Timely response?                      0
Consumer disputed?              1191207
Complaint ID                          0
dtype: int64

In [11]:
data = data.replace(np.nan, '', regex=True)

In [12]:
data.isnull().sum()

Date received                   0
Product                         0
Sub-product                     0
Issue                           0
Sub-issue                       0
Consumer complaint narrative    0
Company public response         0
Company                         0
State                           0
ZIP code                        0
Tags                            0
Consumer consent provided?      0
Submitted via                   0
Date sent to company            0
Company response to consumer    0
Timely response?                0
Consumer disputed?              0
Complaint ID                    0
dtype: int64

In [13]:
data_train = data[['Consumer complaint narrative','Product']]

In [14]:
data_train.shape

(1959680, 2)

In [15]:
data_train.head()

,Consumer complaint narrative,Product
0,transworld systems inc. \nis trying to collect...,Debt collection
1,,"Credit reporting, credit repair services, or o..."
2,"Over the past 2 weeks, I have been receiving e...",Debt collection
3,Pioneer has committed several federal violatio...,Debt collection
4,,Credit card or prepaid card


In [16]:
X = data_train['Consumer complaint narrative']
y = data_train['Product']
y

0                                            Debt collection
1          Credit reporting, credit repair services, or o...
2                                            Debt collection
3                                            Debt collection
4                                Credit card or prepaid card
                                 ...                        
1959675                                        Consumer Loan
1959676                                      Debt collection
1959677                                             Mortgage
1959678                                             Mortgage
1959679                                          Credit card
Name: Product, Length: 1959680, dtype: object

In [17]:
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [18]:
X_train[:30]

842966     Spoke with Executive Financial after I had sen...
165397                                                      
1874482                                                     
43767      ( a ) BlockExcept as otherwise provided in thi...
676462                                                      
1293918                                                     
1959484    I've called the credit bureaus, sent certified...
1030721                                                     
1101923                                                     
719068                                                      
925133     I am filing a complaint against Nationstar/Mr....
1012184                                                     
507523     Equifax - I have sent over 4 dispute letters t...
249517                                                      
1381590                                                     
972459     I have a home loan with Nationstar Mortgage an...
1750799    Continued rob

In [19]:
y_train[:10]

842966                                       Debt collection
165397                           Checking or savings account
1874482    Credit reporting, credit repair services, or o...
43767      Credit reporting, credit repair services, or o...
676462     Credit reporting, credit repair services, or o...
1293918                                      Debt collection
1959484    Credit reporting, credit repair services, or o...
1030721                                     Credit reporting
1101923                                          Credit card
719068                                       Debt collection
Name: Product, dtype: object

#Tokenização de texto

##Com scikit-learn Pré-processamento de texto, tokenização e filtragem de palavras irrelevantes estão todos incluídos no CountVectorizer, que cria um dicionário de recursos e transforma documentos em vetores de recursos:

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(1469760, 127274)

##CountVectorizer suporta contagens de N-gramas de palavras ou caracteres consecutivos. Uma vez ajustado, o vetorizador construiu um dicionário de índices de recursos:

In [21]:
count_vect.vocabulary_.get(u'card')

22997

##O valor do índice de uma palavra no vocabulário está relacionado à sua frequência em todo o corpus de treinamento

##De ocorrências a frequências

##A contagem de ocorrências é um bom começo, mas há um problema: documentos mais longos terão valores médios de contagem mais altos do que documentos mais curtos, embora possam falar sobre os mesmos tópicos. Para evitar essas discrepâncias em potencial, é suficiente dividir o número de ocorrências de cada palavra em um documento pelo número total de palavras no documento: esses novos recursos são chamados de tf para Frequências de termo. Outro refinamento além do tf é reduzir os pesos das palavras que ocorrem em muitos documentos no corpus e, portanto, são menos informativas do que aquelas que ocorrem apenas em uma parte menor do corpus. Essa redução é chamada de tf – idf para “Frequência do termo vezes frequência inversa do documento”. Tanto tf quanto tf – idf podem ser calculados da seguinte maneira usando TfidfTransformer:

In [22]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(1469760, 127274)

##No código de exemplo acima, primeiro usamos o método fit (..) para ajustar nosso estimador aos dados e, em segundo lugar, o método transform (..) para transformar nossa matriz de contagem em uma representação tf-idf. Essas duas etapas podem ser combinadas para atingir o mesmo resultado final mais rápido, ignorando o processamento redundante. Isso é feito usando o método fit_transform (..) conforme mostrado abaixo e conforme mencionado na nota da seção anterior:

In [23]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(1469760, 127274)

#Treinando um classificador 

##Agora que temos nossos recursos, podemos treinar um classificador para tentar prever a categoria de uma postagem. Vamos começar com um classificador Nayve-Bayes, que fornece uma boa base para esta tarefa. O scikit-learn inclui várias variantes desse classificador; o mais adequado para contagem de palavras é a variante multinomial:

In [24]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train)

##Para tentar prever o resultado em um novo documento, precisamos extrair os recursos usando quase a mesma cadeia de extração de recursos de antes. A diferença é que chamamos transform em vez de fit_transform nos transformadores, uma vez que eles já foram ajustados ao conjunto de treinamento:

In [25]:
complaint = ['After sending information to verify myself', 'I previously filed a complaint against Lexis']
X_new_counts = count_vect.transform(complaint)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for comp, category in zip(complaint, predicted):
    print('%r => %s' % (comp, y_train))

'After sending information to verify myself' => 842966                                       Debt collection
165397                           Checking or savings account
1874482    Credit reporting, credit repair services, or o...
43767      Credit reporting, credit repair services, or o...
676462     Credit reporting, credit repair services, or o...
                                 ...                        
837545     Credit reporting, credit repair services, or o...
1255565                                             Mortgage
658501     Credit reporting, credit repair services, or o...
1381819                                             Mortgage
1607022                                      Debt collection
Name: Product, Length: 1469760, dtype: object
'I previously filed a complaint against Lexis' => 842966                                       Debt collection
165397                           Checking or savings account
1874482    Credit reporting, credit repair services, or o...
43

#Construindo um pipeline

## Para tornar o vectorizer => transformer => classifier mais fácil de trabalhar, o scikit-learn fornece uma classe Pipeline que se comporta como um classificador composto:

In [26]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

##Os nomes vect, tfidf e clf (classificador) são arbitrários. Iremos usá-los para realizar o grid search para hiperparâmetros adequados abaixo. Agora podemos treinar o modelo com um único comando:

In [27]:
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

#Avaliação do desempenho no conjunto de teste



In [28]:
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.41326134879163945

##Buscando melhorar performance com máquina de vetores de suporte linear (SVM), que é amplamente considerada como um dos melhores algoritmos de classificação de texto (embora também seja um pouco mais lenta do que o Nayve-Bayes). Podemos mudar o aluno simplesmente conectando um objeto classificador diferente em nosso pipeline:


In [29]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
])

text_clf.fit(X_train,y_train)

predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.4203665904637492

##O scikit-learn fornece mais utilitários para uma análise de desempenho mais detalhada dos resultados:

In [31]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted))

                                                                              precision    recall  f1-score   support

                                                     Bank account or service       0.77      0.00      0.01     21763
                                                 Checking or savings account       0.57      0.16      0.25     20329
                                                               Consumer Loan       0.33      0.00      0.01      8041
                                                                 Credit card       0.54      0.00      0.01     22369
                                                 Credit card or prepaid card       0.60      0.18      0.27     25518
                                                            Credit reporting       0.23      0.00      0.00     35437
Credit reporting, credit repair services, or other personal consumer reports       0.37      0.98      0.54    158844
                                                       

In [32]:
metrics.confusion_matrix(y_test, predicted)

array([[   101,   1580,      3,     12,    234,      3,  18821,    351,
            69,      1,    558,      0,      1,      5,      6,     14,
             2,      2],
       [     5,   3223,      1,      1,    367,      0,  15321,    594,
           108,      0,    677,      0,      1,      7,      0,     19,
             2,      3],
       [     0,     21,     28,      4,     36,      0,   6496,    608,
             5,      0,    572,      0,      1,     23,      0,     80,
           167,      0],
       [    10,     44,      7,     58,   1653,      2,  19545,    758,
            20,      0,    230,      0,      1,      3,      1,     35,
             2,      0],
       [     1,    189,      3,     11,   4471,      0,  18826,   1309,
            49,      1,    601,      2,      2,      7,      0,     37,
             5,      4],
       [     0,      4,      0,      0,      8,      5,  34867,    470,
             1,      0,     73,      1,      0,      0,      0,      6,
           